# Проект: классификация

In [232]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing  import LabelEncoder
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif

## Часть 1. Знакомство с данными, обработка пропусков и выбросов

### Задание 1

In [233]:
df = pd.read_csv('data/bank_fin.csv', sep = ';')

In [234]:
# исследуйте данные на предмет пропусков. Где есть пропущенные значения? Сколько их?

display(df.sample(3))
df.isnull().sum()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
5089,36,blue-collar,divorced,secondary,no,"2 843,00 $",no,no,cellular,14,aug,186,1,-1,0,unknown,yes
10838,44,housemaid,single,secondary,no,"299,00 $",no,no,cellular,20,nov,186,1,-1,0,unknown,no
4452,41,blue-collar,married,secondary,no,"2 689,00 $",yes,no,cellular,15,apr,135,1,-1,0,unknown,yes


,0
age,0
job,0
marital,0
education,0
default,0
balance,25
housing,0
loan,0
contact,0
day,0


### Задание 2

In [235]:
# есть ли в признаке job пропущенные значения? Возможно, они обозначены каким-то специальным словом?
df['job'].value_counts()

,count
job,
management,2566
blue-collar,1944
technician,1823
admin.,1334
services,923
retired,778
self-employed,405
student,360
unemployed,357


### Задание 3

In [236]:
# преобразуйте признак balance таким образом, чтобы он корректно считывался, как вещественное число (float)
def balance_to_float(x):
    x = str(x)
    return float(x.split(',')[0].replace(' ', '').replace('$', ''))

df['balance'] = df['balance'].apply(balance_to_float)


### Задание 4

In [237]:
# обработайте пропуски в признаки balance , заменив их на медианные значения по данному признаку

df.loc[df['balance'].isnull(), 'balance'] = df['balance'].median()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        11162 non-null  int64  
 1   job        11162 non-null  object 
 2   marital    11162 non-null  object 
 3   education  11162 non-null  object 
 4   default    11162 non-null  object 
 5   balance    11162 non-null  float64
 6   housing    11162 non-null  object 
 7   loan       11162 non-null  object 
 8   contact    11162 non-null  object 
 9   day        11162 non-null  int64  
 10  month      11162 non-null  object 
 11  duration   11162 non-null  int64  
 12  campaign   11162 non-null  int64  
 13  pdays      11162 non-null  int64  
 14  previous   11162 non-null  int64  
 15  poutcome   11162 non-null  object 
 16  deposit    11162 non-null  object 
dtypes: float64(1), int64(6), object(10)
memory usage: 1.4+ MB


In [238]:
df['job'].value_counts()

,count
job,
management,2566
blue-collar,1944
technician,1823
admin.,1334
services,923
retired,778
self-employed,405
student,360
unemployed,357


### Задание 5

In [239]:
# обработайте пропуски в категориальных признаках: job и education, заменив их на модальные значения

df.loc[df['job'] == 'unknown', 'job'] = df['job'].mode()[0]
df.loc[df['education'] == 'unknown', 'education'] = df['education'].mode()[0]

# np.round(df[(df['job'] == df['job'].value_counts().reset_index()['job'][0]) & (df['education'] == df['education'].value_counts().reset_index()['education'][0])]['balance'].mean(), 3)



np.float64(1598.883)

### Задание 6

In [277]:
# удалите все выбросы для признака balance
def tyuki(df, col):
   x = df[col]
   q1 = x.quantile(0.25)
   q3 = x.quantile(0.75)
   iqr = q3 - q1
   lower = q1- 1.5*iqr
   higher = q3 + 1.5*iqr
   outliers = x[(x < (lower)) | (x > (higher))]
   cleaned = x[(x >= (lower)) & (x <= (higher))]
   return cleaned, outliers, lower, higher


cleaned, outliers, lower, higher  = tyuki(df, 'balance')

print("Outliers count:", len(outliers))
print("Cleaned count:", len(cleaned))
print("Lower bound:", lower)
print("Upper bound:", higher)

Outliers count: 1057
Cleaned count: 10105
Lower bound: -2241.0
Upper bound: 4063.0


## Часть 2:  Разведывательный анализ

### Задание 1

In [241]:
# изучите соотношение классов в ваших данных на предмет несбалансированности, проиллюстрируйте результат
# ваш код

### Задания 2 и 3

In [242]:
#рассчитайте описательные статистики для количественных переменных, проинтерпретируйте результат
#ваш код

### Задания 4 и 5

In [243]:
#рассчитайте описательные статистики для категориальных переменных, проинтерпретируйте результат
#ваш код
#постройте визуализации, иллюстрирующие результаты

### Задание 6

In [244]:
# Узнайте, для какого статуса предыдущей маркетинговой кампании успех в текущей превалирует над количеством неудач.
# ваш код

### Задание 7

In [245]:
# узнайте, в каком месяце чаще всего отказывались от предложения открыть депозит
# ваш код

### Задание 8

In [246]:
# создайте возрастные группы и определите, в каких группах более склонны открывать депозит, чем отказываться от предложения

### Задания 9 и 10

In [247]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от семейного статуса

In [248]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от образования

In [249]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от вида профессиональной занятости

### Задание 11

In [250]:
# постройте сводную таблицу, чтобы определить люди с каким образованием и семейным статусом наиболее многочисленны
#(если рассматривать тех, кто открыл депозит)

## Часть 3: преобразование данных

### Задание 1

In [251]:
# преобразуйте уровни образования

### Задания 2 и 3

In [252]:
# преобразуйте бинарные переменные в представление из нулей и единиц

### Задание 4

In [253]:
# создайте дамми-переменные

### Задания 5 и 6

In [254]:
# постройте корреляционную матрицу и оцените данные на предмет наличия мультиколлинеарности

### Задания 7 и 8

In [255]:
X = df.drop(['deposit'], axis=1)
y = df['deposit']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.33)

In [256]:
# рассчитайте необходимые показатели

### Задание 9

In [257]:
# с помощью SelectKBest отберите 15 наиболее подходящих признаков

### Задание 10

In [258]:
# нормализуйте данные с помощью minmaxsxaler

# Часть 4: Решение задачи классификации: логистическая регрессия и решающие деревья

### Задание 1

In [259]:
# обучите логистическую регрессию и рассчитайте метрики качества

### Задания 2,3,4

In [260]:
# обучите решающие деревья, настройте максимальную глубину

### Задание 5

In [261]:
# подберите оптимальные параметры с помощью gridsearch

# Часть 5: Решение задачи классификации: ансамбли моделей и построение прогноза

### Задание 1

In [262]:
# обучите на ваших данных случайный лес

### Задания 2 и 3

In [263]:
# используйте для классификации градиентный бустинг и сравните качество со случайным лесом

### Задание 4

In [264]:
# объедините уже известные вам алгоритмы с помощью стекинга

### Задание 5

In [265]:
# оцените, какие признаки демонстрируют наибольшую  важность в модели градиентного бустинга

### Задания 6,7,8

In [266]:
# реализуйте оптимизацию гиперпараметров с помощью Optuna